In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import string
import argparse
import time
import os
from six.moves import cPickle
from TextLoader import *
from Hangulpy import *
print ("Packages Imported")

Packages Imported


In [2]:
## Load dataset using TextLoader
data_dir    = "data/"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "data/nine_dreams"   
#  from "data/nine_dreams/input.txt" 
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "\n" )
print ("data_loader.vocab looks like \n%s " %
       (data_loader.vocab))
print ( "\n" )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )
print ( "\n" )
print ("data_loader.chars looks like \n%s " % (data_loader.chars,))

loading preprocessed files
type of 'data_loader' is <class 'dict'>, length is 75


data_loader.vocab looks like 
{'ᴥ': 0, ' ': 1, 'ㅇ': 2, 'ㅏ': 3, 'ㄴ': 4, 'ㄹ': 5, 'ㅣ': 6, 'ㄱ': 7, 'ㅡ': 8, 'ㅗ': 9, 'ㅅ': 10, 'ㅓ': 11, 'ㅁ': 12, 'ㄷ': 13, 'ㅎ': 14, 'ㅈ': 15, 'ㅜ': 16, 'ㅂ': 17, 'ㅕ': 18, '\n': 19, 'ㅔ': 20, 'ㅐ': 21, 'ㅆ': 22, 'ㅊ': 23, '.': 24, 'ㅢ': 25, 'ㅘ': 26, ',': 27, '"': 28, 'ㄲ': 29, 'ㅌ': 30, 'ㅍ': 31, 'ㄸ': 32, 'ㅚ': 33, 'ㅑ': 34, 'ㅟ': 35, 'ㅝ': 36, 'ㅠ': 37, 'ㅛ': 38, 'ㅖ': 39, 'ㅉ': 40, '?': 41, 'ㅋ': 42, 'ㅄ': 43, 'ㄶ': 44, '(': 45, ')': 46, 'ㅃ': 47, 'ㄺ': 48, "'": 49, 'ㄵ': 50, 'ㅀ': 51, '!': 52, 'ㅙ': 53, 'ㄼ': 54, 'ㄻ': 55, 'ㄿ': 56, ':': 57, '1': 58, '6': 59, '2': 60, 'ㄳ': 61, 'ㅞ': 62, '9': 63, '5': 64, '-': 65, '4': 66, '3': 67, '8': 68, '7': 69, '_': 70, 'ㄾ': 71, 'ㅒ': 72, '0': 73, '>': 74} 


type of 'data_loader.chars' is <class 'tuple'>, length is 75


data_loader.chars looks like 
('ᴥ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', 'ㄹ', 'ㅣ', 'ㄱ', 'ㅡ', 'ㅗ', 'ㅅ', 'ㅓ', 'ㅁ', 'ㄷ', 'ㅎ', 'ㅈ', 'ㅜ', 'ㅂ', 'ㅕ', '\n', 'ㅔ', 'ㅐ', 'ㅆ', 'ㅊ', '.

In [3]:
## Define network
rnn_size   = 512
num_layers = 3
grad_clip  = 5.

_batch_size = 1
_seq_length = 1

vocab_size = data_loader.vocab_size

with tf.device("/cpu:0"):
    # Select RNN Cell
    unitcell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([unitcell] * num_layers)
    # Set paths to the graph 
    input_data = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    targets    = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    initial_state = cell.zero_state(_batch_size, tf.float32)

    # Set Network
    with tf.variable_scope('rnnlm'):
        softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        with tf.device("/cpu:0"):
            embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
            inputs = tf.split(tf.nn.embedding_lookup(embedding, input_data), _seq_length, 1)
            inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
            
    # Loop function for seq2seq
    def loop(prev, _):
        prev = tf.nn.xw_plus_b(prev, softmax_w, softmax_b)
        prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
        return tf.nn.embedding_lookup(embedding, prev_symbol)
    # Output of RNN 
    outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state
                                , cell, loop_function=None, scope='rnnlm')
#     output = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
    output = tf.reshape(outputs, [-1, rnn_size])
    logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
    # Next word probability 
    probs = tf.nn.softmax(logits)
    # Define LOSS
    loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], # Input
        [tf.reshape(targets, [-1])], # Target
        [tf.ones([_batch_size * _seq_length])], # Weight 
        vocab_size)
    # Define Optimizer
    cost = tf.reduce_sum(loss) / _batch_size / _seq_length
    final_state = last_state
    lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    _optm = tf.train.AdamOptimizer(lr)
    optm = _optm.apply_gradients(zip(grads, tvars))

print ("Network Ready")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Network Ready


In [4]:
## Sample!!

def sample( sess, chars, vocab, __probs, num=200, prime=u'ㅇㅗᴥㄴㅡㄹᴥ '):
    state = sess.run(cell.zero_state(1, tf.float32))
    _probs = __probs
    prime = list(prime)
    for char in prime[:-1]:
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [state] = sess.run([final_state], feed)

    def weighted_pick(weights):
        weights = weights / np.sum(weights) 
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime
    char = prime[-1]
    for n in range(num):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [_probsval, state] = sess.run([_probs, final_state], feed)
        p = _probsval[0]
        sample = int(np.random.choice(len(p), p=p))
        # sample = weighted_pick(p)
        # sample = np.argmax(p)
        pred = chars[sample]
        ret += pred
        char = pred
    return ret
print ("sampling function done.")

sampling function done.


## Sample

In [5]:
save_dir = 'data/nine_dreams'
prime = decompose_text(u"누구 ")

print ("Prime Text : %s => %s" % (automata(prime), "".join(prime)))
n = 2000

sess = tf.Session()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
ckpt = tf.train.get_checkpoint_state(save_dir)

# load_name = u'data/nine_dreams/model.ckpt-0'
load_name = u'data/nine_dreams/model.ckpt-99000'

print (load_name)

if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, load_name)
    sampled_text = sample(sess, chars, vocab, probs, n, prime)
    #print ("")
    print (u"SAMPLED TEXT = %s" % sampled_text)
    print ("")
    print ("-- RESULT --")
    print (automata("".join(sampled_text)))

Prime Text : 누구  => ㄴㅜᴥㄱㅜᴥ 
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
data/nine_dreams/model.ckpt-99000
INFO:tensorflow:Restoring parameters from data/nine_dreams/model.ckpt-99000
SAMPLED TEXT = ['ㄴ', 'ㅜ', 'ᴥ', 'ㄱ', 'ㅜ', 'ᴥ', ' ', 'ᴥ', 'ㅡ', 'ㅎ', 'ㅣ', 'ㅏ', 'ㄴ', ' ', 'ㄱ', 'ᴥ', 'ᴥ', 'ㅓ', 'ㅇ', 'ㄱ', 'ㅏ', 'ᴥ', 'ᴥ', 'ㄱ', 'ㄴ', 'ᴥ', 'ᴥ', ' ', ' ', ' ', ' ', 'ㅇ', 'ㅣ', ' ', 'ㄴ', ' ', 'ᴥ', 'ㅓ', 'ㅅ', 'ᴥ', 'ㅓ', 'ㅢ', 'ㅓ', 'ㅇ', 'ㅇ', 'ㅈ', 'ᴥ', 'ㅇ', 'ㅇ', 'ᴥ', 'ㅎ', 'ᴥ', 'ᴥ', 'ㄸ', 'ㅅ', ' ', 'ㅡ', 'ㄹ', 'ㅇ', 'ㄴ', 'ㅇ', 'ᴥ', 'ᴥ', 'ㄱ', 'ㅅ', ' ', 'ㅣ', 'ᴥ', 'ᴥ', 'ᴥ', 'ᴥ', 'ㅇ', 'ᴥ', 'ㅇ', 'ㅗ', 'ㄹ', ' ', 'ᴥ', 'ㅇ', 'ㅣ', 'ᴥ', 'ᴥ', 'ㅊ', 'ㄱ', 'ᴥ', 'ㄷ', 'ㄱ', 'ㅘ', 'ᴥ', 'ㅔ', 'ㄴ', 'ㄴ', 'ㅇ', 'ᴥ', 'ㄷ', 'ㅇ', 'ᴥ', 'ㄱ', 'ㅅ', 'ㅅ', 'ㅡ', 'ㅇ', 'ㅓ', 'ㅜ', 'ㅣ', 'ㄹ', 'ㅗ', 'ᴥ', 'ㅔ', 'ㅘ', 'ㅗ', 'ㄱ', 'ᴥ', 'ᴥ', 'ᴥ', 'ㄱ', 'ㄱ', 'ㅛ', 'ㅇ', 'ㅡ', 'ㅅ', 'ᴥ', 'ㅘ', 'ㅆ', ' ', 'ㅇ', 'ㅐ', 'ㅡ', 'ᴥ', 'ㄴ', 'ᴥ', 'ㅁ', 'ㅇ', 'ᴥ', 'ᴥ', ' ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅓ', 'ㅔ', 